In [1]:
import json
import os
import SharedArray as sa
from typing import Optional
from datetime import datetime
import json
import random
import numpy as np
import torch

from alphagen.data.expression import *
from alphagen_ocean.calculator_ import QLibStockDataCalculator
from alphagen_ocean.stock_data_ import StockData
from alphagen.config import *
from alphagen.data.tree import ExpressionBuilder
from alphagen.data.tokens import *

from utils import formula_to_expression

In [2]:
device = torch.device("cpu")
cuda = torch.device("cuda:3")
data_test = StockData(
    start_time=20210101,
    end_time=20210601,
    device=device,
)

In [3]:
path = "checkpoints/ocean_lexpr8_lopt34_10_962_20230811101530/251904_steps_pool.json"
with open(path, "r") as f:
    alpha = json.load(f)

In [4]:
factors = [0] * 10
for i in range(10):
    if alpha["exprs"][i][0] == "$":
        factors[i] = Feature(getattr(FeatureType, alpha["exprs"][i][1:])).evaluate(
            data_test
        )
    else:
        factors[i] = formula_to_expression(alpha["exprs"][i]).evaluate(data_test)

In [5]:
weights = torch.Tensor(alpha["weights"])

In [6]:
factor_value = torch.zeros_like(factors[0])
for i in range(10):
    factor_value += factors[i] * weights[i]

In [7]:
from alphagen.utils.pytorch_utils import normalize_by_day

In [8]:
yhat = normalize_by_day(factor_value)

In [9]:
cal_test = QLibStockDataCalculator(data_test)

mem of retx:65.093994140625 MB


In [10]:
y1d = cal_test.ret1d
y2d = cal_test.ret2d
y5d = cal_test.ret5d

In [11]:
from alphagen.utils.correlation import batch_pearsonr, batch_spearmanr

In [20]:
for y in [y1d, y2d, y5d]:
    print(torch.nanmean(batch_pearsonr(yhat, y)).item())
    # print(torch.mean(batch_spearmanr(yhat, y)))

0.01371405838093669
0.016069599268724873
0.016069599268724873


In [14]:
import torch as th


def calc_topk(input: Tensor, target: Tensor, topk: int = 10) -> Tensor:
    idx = ~th.isnan(target)
    input, target = input[idx], target[idx]
    top_q_indices = th.topk(input, int(len(target) * topk / 100), largest=True).indices
    q_ret = th.nanmean(target[top_q_indices])
    return q_ret

In [18]:
q90 = []
for i in range(len(yhat)):
    q90.append(calc_topk(yhat[i], y[i], 10).item())
np.mean(q90)

0.0005885508181959288

In [19]:
q99 = []
for i in range(len(yhat)):
    q99.append(calc_topk(yhat[i], y[i], 1).item())
np.mean(q99)

0.00028853024088446966

In [22]:
def nan_pearsonr(input, target):
    nan_mask = ~th.isnan(target)
    input, target = input[nan_mask], target[nan_mask]
    return th.corrcoef(th.stack([input, target]))[0, 1]


ic = []
for i in range(len(yhat)):
    ic.append(nan_pearsonr(yhat[i], y[i]))
np.mean(ic)

0.016069599288809938